ResNet pretrained

In [ ]:
# Import libraries
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os

# PyTorch dataset
from torchvision import datasets
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

# PyTorch model
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


from sklearn.metrics import precision_recall_fscore_support
import time

In [ ]:
torch.manual_seed(42)

In [ ]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20

In [ ]:
train_transform =  transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

test_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.images = self.load_images()

    def load_images(self):
        images = []
        for cls in self.classes:
            class_dir = os.path.join(self.root_dir, cls)
            for filename in os.listdir(class_dir):
                if filename.endswith(".png"):
                    path = os.path.join(class_dir, filename)
                    label = self.class_to_idx[cls]
                    images.append((path, label))
        return images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
root_dir = "Data_20/train"
custom_dataset_train = CustomDataset(root_dir, transform = train_transform)

In [ ]:
root_dir = "Data_20/val"
custom_dataset_val = CustomDataset(root_dir, transform = test_transform)

In [ ]:
root_dir = "Data_20/test"
custom_dataset_test = CustomDataset(root_dir, transform = test_transform)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(custom_dataset_train, batch_size = batch_size, shuffle = True)

In [ ]:
val_dataloader = torch.utils.data.DataLoader(custom_dataset_val, batch_size = batch_size, shuffle = True)

In [ ]:
test_dataloader = torch.utils.data.DataLoader(custom_dataset_test, batch_size = batch_size, shuffle = True)

In [ ]:
# specify the image classes
classes = ['Bycicle', 'Bus', 'Motorcycle', 'Pedestrian', 'Private_car', 'Taxi_or_uber']
classes

In [ ]:
def imshow(img):
    img = np.transpose(img, (1, 2, 0)) 
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    img = np.clip(img, 0, 1)
    plt.imshow(img) 

In [ ]:
dataiter = iter(train_dataloader)
images, labels = next(dataiter)
images = images.numpy() 
images.shape #

In [ ]:
fig = plt.figure(figsize = (25, 10))

for idx in np.arange(10):
    ax = fig.add_subplot(2, int(10/2), idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(classes[labels[idx]])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

cudnn.benchmark = True
plt.ion()

In [ ]:
model = models.resnet18(weights = 'IMAGENET1K_V1')

in_features = model.fc.in_features
out_features = len(classes) 

model.fc = nn.Linear(in_features, out_features)

In [ ]:
if train_on_gpu:
    model.cuda()

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters())

In [ ]:
train_loss_f = []
val_loss_f = []


acc_train = []
acc_val = []


recall_train = []
recall_val = []


prec_train = []
prec_val = []


F1_train = []
F1_val = []

Time_list = []
time_count = 0.0

n_epochs = 50

valid_loss_min = np.Inf 

for epoch in range(1, n_epochs+1):

    train_loss = 0.0
    valid_loss = 0.0
    
    running_correct_train = 0.0
    running_correct_val = 0.0
    ###################
    # train the model #
    ###################

    y_true = []
    y_pred = []

    start_time = time.time()
    
    model.train()
    for data, target in train_dataloader:

        if train_on_gpu:
            data, target = data.cuda(), target.cuda()


        optimizer.zero_grad()

        output = model(data)
        
        loss = criterion(output, target)

        _, pred = torch.max(output, 1)

        loss.backward()
        

        optimizer.step()
        

        train_loss += loss.item()*data.size(0)
        
        correct =  torch.sum(pred == target).cpu().numpy()
        running_correct_train += correct

        y_true = y_true + target.cpu().numpy().tolist()
        y_pred = y_pred + pred.cpu().numpy().tolist()

    prec_train.append(precision_recall_fscore_support(y_true, y_pred, average = 'micro')[0])
    recall_train.append(precision_recall_fscore_support(y_true, y_pred, average = 'micro')[1])
    F1_train.append(precision_recall_fscore_support(y_true, y_pred, average = 'micro')[2])

    
    ######################    
    # validate the model #
    ######################
    y_true = []
    y_pred = []
    
    model.eval()
    for data, target in val_dataloader:

        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
            
        output = model(data)
        _, pred = torch.max(output, 1)
        
        loss = criterion(output, target)

        valid_loss += loss.item()*data.size(0)
        
        correct = torch.sum(pred == target).cpu().numpy()
        running_correct_val += correct
        
        y_true = y_true + target.cpu().numpy().tolist()
        y_pred = y_pred + pred.cpu().numpy().tolist()

    end_time = time.time()
    time_count += end_time - start_time
    Time_list.append(time_count)

    prec_val.append(precision_recall_fscore_support(y_true, y_pred, average = 'micro')[0])
    recall_val.append(precision_recall_fscore_support(y_true, y_pred, average = 'micro')[1])
    F1_val.append(precision_recall_fscore_support(y_true, y_pred, average = 'micro')[2])


    train_loss = train_loss/len(train_dataloader.sampler)
    train_loss_f.append(train_loss)
    
    valid_loss = valid_loss/len(val_dataloader.sampler)
    val_loss_f.append(valid_loss)

    acc_train.append(running_correct_train / len(train_dataloader.sampler))
    acc_val.append(running_correct_val / len(val_dataloader.sampler))
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_R_20_1.pt')
        valid_loss_min = valid_loss

In [ ]:
plt.figure(figsize = (4, 6))
plt.ylim([0, 1])
plt.xlim([0, n_epochs + 1])
plt.plot(range(1, n_epochs + 1, 1), acc_train, label = "Training Acc", linewidth = 0.5)
plt.plot(range(1, n_epochs + 1, 1), acc_val, label = "Validation Acc", linewidth = 0.5)
plt.title('Training and Validation Accuracy')
plt.legend(loc = "upper right")
plt.ylabel('Acc')
plt.grid(color = 'lightsteelblue', linestyle = '-', linewidth = 0.12)
plt.show()

In [ ]:
plt.figure(figsize = (4, 5))
plt.ylim([-0.2, 2])
plt.xlim([0, n_epochs + 1])
plt.plot(range(1, n_epochs + 1, 1), train_loss_f, label = "Trainig Loss", linewidth = 0.5)
plt.plot(range(1, n_epochs + 1, 1), val_loss_f, label = "Validation Loss", linewidth = 0.5)
plt.title('Training and Validation Loss')
plt.legend(loc = "upper right")
plt.ylabel('Loss')
plt.grid(color = 'lightsteelblue', linestyle = '-', linewidth = 0.12)
plt.show()

In [ ]:
model.load_state_dict(torch.load('model_R_20_1.pt'))

In [ ]:
test_loss = 0.0

class_correct = list(0. for i in range(len(classes)))
class_total = list(0. for i in range(len(classes)))
class_total_pred = list(0. for i in range(len(classes)))

y_true = []
y_pred = []

model.eval()


for data, target in test_dataloader:

    if train_on_gpu:
        data, target = data.cuda(), target.cuda()

    output = model(data)

    loss = criterion(output, target)

    test_loss += loss.item()*data.size(0)

    _, pred = torch.max(output, 1)    

    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()).reshape(-1) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy()).reshape(-1)

    y_true = y_true + target.cpu().numpy().tolist()
    y_pred = y_pred + pred.cpu().numpy().tolist()
    

    for i in range(len(target)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

    for i in range(len(pred)):
        label = pred.data[i]
        class_total_pred[label] += 1

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
con = confusion_matrix(y_true, y_pred, labels = range(len(classes)))
disp = ConfusionMatrixDisplay(con,  display_labels = classes)

In [ ]:
disp.plot(xticks_rotation = 45)
plt.show()

In [ ]:
test_loss = test_loss/len(test_dataloader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(len(classes)):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

overall_correct = sum(class_correct)
overall_total = sum(class_total)
print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * overall_correct / overall_total,
    overall_correct, overall_total))

In [ ]:
import pandas as pd

In [ ]:
col = {'epoch': range(1, n_epochs + 1, 1), 'acc_train': acc_train, 'acc_val': acc_val, 'loss_train': train_loss_f, 'loss_val': val_loss_f, 'pre_train': prec_train, 'pre_val': prec_val, 'rec_train': recall_train, 'rec_val': recall_val, 'F1_train': F1_train, 'F1_val': F1_val, 'Time': Time_list}
df_loss_acc = pd.DataFrame(col)

col = {'y_true': y_true, 'y_pred': y_pred}
df_conf_mat = pd.DataFrame(col)

In [ ]:
df_loss_acc.to_csv('metriche_R_20_1.csv', index = False)
df_conf_mat.to_csv('confusion_matrix_R_20_1.csv', index = False)